In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import datetime
import cartopy.crs as ccrs
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.models.emissions import Emissions
import boxm_for

from pycontrails.physics import units
from pycontrails.core.met_var import (
    AirTemperature,
    SpecificHumidity,
)
from init_chem import CHEM
from chem import ChemDataset
from boxm import BoxModel

In [2]:
# Initialise coord arrays
lon_bounds = (0, 5) #np.arange(-180, 180, 5)
lat_bounds = (0, 5) #np.arange(-90, 90, 5)
alt_bounds = (8000, 12000)
horiz_res = 2.5
vert_res = 500
met_pressure_levels = np.array([400, 300, 200, 100])

time = ("2022-01-01 00:00:00", "2022-01-01 08:00:00")
ts_met = "1H"
ts_disp = "1min"
ts_chem = "1H"

runtime = "24H"

# lons = [0]
# lats = [0]
# alts = [11000]

lons = np.arange(lon_bounds[0], lon_bounds[1], horiz_res)
lats = np.arange(lat_bounds[0], lat_bounds[1], horiz_res)
alts = np.arange(alt_bounds[0], alt_bounds[1], vert_res)

In [3]:
# Import met data from ERA5
era5 = ERA5(
        time=time,
        timestep_freq=ts_met,
        variables=[
                "t",
                "q",
                "u",
                "v",
                "w",
                "z",
                "relative_humidity"
        ],
        pressure_levels=met_pressure_levels
)

In [4]:
# download data from ERA5 (or open from cache)
met = era5.open_metdataset()
met.data = met.data.transpose("latitude", "longitude", "level", "time", ...)
met


MetDataset with data:

<xarray.Dataset>
Dimensions:                              (latitude: 721, longitude: 1440,
                                          level: 4, time: 9)
Coordinates:
  * longitude                            (longitude) float64 -180.0 ... 179.8
  * latitude                             (latitude) float64 -90.0 ... 90.0
  * level                                (level) float64 100.0 200.0 300.0 400.0
  * time                                 (time) datetime64[ns] 2022-01-01 ......
    air_pressure                         (level) float32 1e+04 2e+04 3e+04 4e+04
    altitude                             (level) float32 1.618e+04 ... 7.185e+03
Data variables:
    air_temperature                      (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    specific_humidity                    (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    eastward_wind                        (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    northward_wind                       (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    lagrangian_tendency_of_air_pressure  (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    geopotential                         (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    relative_humidity                    (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
Attributes:
    Conventions:          CF-1.6
    history:              2023-09-20 12:15:22 GMT by grib_to_netcdf-2.25.1: /...
    pycontrails_version:  0.47.0
    met_source:           ERA5

In [5]:
# Initialise low-res chem xarray dataset, and import species files
init_chem = CHEM(
        time=time,
        timestep_freq=ts_chem,
)

init_chem.timesteps

[datetime.datetime(2022, 1, 1, 0, 0),
 datetime.datetime(2022, 1, 1, 1, 0),
 datetime.datetime(2022, 1, 1, 2, 0),
 datetime.datetime(2022, 1, 1, 3, 0),
 datetime.datetime(2022, 1, 1, 4, 0),
 datetime.datetime(2022, 1, 1, 5, 0),
 datetime.datetime(2022, 1, 1, 6, 0),
 datetime.datetime(2022, 1, 1, 7, 0),
 datetime.datetime(2022, 1, 1, 8, 0)]

In [6]:
# Populate chem dataset with species data
chem = init_chem.open_chemdataset()

<xarray.Dataset>
Dimensions:        (latitude: 36, longitude: 72, time: 9, photol_params: 34,
                    level: 9, photol_coeffs: 96, therm_coeffs: 510, species: 219)
Coordinates:
  * latitude       (latitude) float64 -90.0 -85.0 -80.0 -75.0 ... 75.0 80.0 85.0
  * longitude      (longitude) float64 -180.0 -175.0 -170.0 ... 170.0 175.0
  * level          (level) float64 962.0 861.0 759.0 658.0 ... 353.0 251.0 150.5
  * time           (time) datetime64[ns] 2022-01-01 ... 2022-01-01T08:00:00
  * photol_params  (photol_params) float64 1.0 2.0 3.0 4.0 ... 55.0 56.0 57.0
  * photol_coeffs  (photol_coeffs) int64 1 2 3 4 5 6 7 ... 90 91 92 93 94 95 96
  * therm_coeffs   (therm_coeffs) int64 1 2 3 4 5 6 ... 505 506 507 508 509 510
  * species        (species) <U10 'O1D' 'O' 'OH' ... 'CH3O2NO2' 'EMPOA' 'P2007'
Data variables:
    local_time     (latitude, longitude, time) float64 dask.array<chunksize=(36, 72, 1), meta=np.ndarray>
    sza            (latitude, longitude, time) float64 da

In [7]:
met.data = met.data.interp(longitude=lons, latitude=lats, level=units.m_to_pl(alts), method="linear")

chem.data = chem.data.interp(longitude=lons, latitude=lats, level=units.m_to_pl(alts), method="linear")

In [8]:
chem.zenith()
chem.get_photol_params()

<xarray.DataArray 'sza' (latitude: 2, longitude: 2, time: 9, photol_params: 34)>
array([[[[2.7480331 , 2.7480331 , 2.7480331 , ..., 2.7480331 ,
          2.7480331 , 2.7480331 ],
         [2.67272536, 2.67272536, 2.67272536, ..., 2.67272536,
          2.67272536, 2.67272536],
         [2.49778773, 2.49778773, 2.49778773, ..., 2.49778773,
          2.49778773, 2.49778773],
         ...,
         [1.57079633, 1.57079633, 1.57079633, ..., 1.57079633,
          1.57079633, 1.57079633],
         [1.32942718, 1.32942718, 1.32942718, ..., 1.32942718,
          1.32942718, 1.32942718],
         [1.09080109, 1.09080109, 1.09080109, ..., 1.09080109,
          1.09080109, 1.09080109]],

        [[2.74574718, 2.74574718, 2.74574718, ..., 2.74574718,
          2.74574718, 2.74574718],
         [2.64835644, 2.64835644, 2.64835644, ..., 2.64835644,
          2.64835644, 2.64835644],
         [2.46372872, 2.46372872, 2.46372872, ..., 2.46372872,
          2.46372872, 2.46372872],
...
         [1.58752

/home/ktait98/pycontrails_kt/pycontrails/models/plume_chem/chem.py:44: UserWarning: Converting non-nanosecond precision timedelta values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  delta = offsets.astype('timedelta64[s]')


In [9]:
# p = chem["sza"].data.sel(time=chem.data.time[0]).plot(
#     subplot_kws=dict(projection=ccrs.Orthographic(20, 10), facecolor="gray"),
#     transform=ccrs.PlateCarree(),
# )

# p.axes.set_global()
# p.axes.coastlines()

In [10]:
# chem_plot = chem.data["Y"].sel(species="NO", time=time[0], level=chem.data.level[0]).plot(
#     subplot_kws=dict(projection=ccrs.Orthographic(-50, 10), facecolor="gray"),
#     transform=ccrs.PlateCarree(),
# )

# chem_plot.axes.set_global()
# chem_plot.axes.coastlines()

In [11]:
# import cartopy.crs as ccrs

# p = met.data["air_temperature"].isel(level=0, time=0).plot(
#     subplot_kws=dict(projection=ccrs.Orthographic(20, 10), facecolor="gray"),
#     transform=ccrs.PlateCarree(),
# )

# p.axes.set_global()

# p.axes.coastlines()

# # save figure
# p.figure.savefig(str(lon_bounds) + str(lat_bounds) + ".png")

# type(p)

In [12]:
# chem.data["Y"].sel(species="DUMMY2", time=time[0], level=962, method="nearest").plot()

In [13]:
boxm = BoxModel(met, chem)

/home/ktait98/miniconda3/envs/traffic/lib/python3.10/site-packages/pycontrails/core/models.py:204: UserWarning: 
Met data appears to have originated from ECMWF and no humidity scaling is enabled. For ECMWF data, consider using one of: 
 - 'ConstantHumidityScaling'
 - 'ExponentialBoostHumidityScaling'
 - 'ExponentialBoostLatitudeCorrectionHumidityScaling'
 - 'HistogramMatching'
For example: 
>>> from pycontrails.models.humidity_scaling import ConstantHumidityScaling
>>> BoxModel(met=met, ..., humidity_scaling=ConstantHumidityScaling(rhi_adj=0.99))
  warnings.warn(
/home/ktait98/miniconda3/envs/traffic/lib/python3.10/site-packages/pycontrails/core/met.py:745: UserWarning: Overwriting data in keys `['air_pressure']`. Use `.update(...)` to suppress warning.
  warnings.warn(


In [14]:
boxm.met

MetDataset with data:

<xarray.Dataset>
Dimensions:                              (latitude: 2, longitude: 2, level: 8,
                                          time: 9)
Coordinates:
  * time                                 (time) datetime64[ns] 2022-01-01 ......
    air_pressure                         (latitude, longitude, level, time) float64 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
    altitude                             (level) float64 8.056e+03 ... 1.154e+04
  * longitude                            (longitude) float64 0.0 2.5
  * latitude                             (latitude) float64 0.0 2.5
  * level                                (level) float64 356.0 331.0 ... 209.2
Data variables:
    air_temperature                      (latitude, longitude, level, time) float32 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
    specific_humidity                    (latitude, longitude, level, time) float32 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
    eastward_wind                        (latitude, longitude, level, time) float32 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
    northward_wind                       (latitude, longitude, level, time) float32 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
    lagrangian_tendency_of_air_pressure  (latitude, longitude, level, time) float32 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
    geopotential                         (latitude, longitude, level, time) float32 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
    relative_humidity                    (latitude, longitude, level, time) float32 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
Attributes:
    Conventions:          CF-1.6
    history:              2023-09-20 12:15:22 GMT by grib_to_netcdf-2.25.1: /...
    pycontrails_version:  0.47.0
    met_source:           ERA5

In [15]:
boxm.chem

ChemDataset with data:

<xarray.Dataset>
Dimensions:        (latitude: 2, longitude: 2, time: 9, photol_params: 34,
                    level: 8, photol_coeffs: 96, therm_coeffs: 510, species: 219)
Coordinates:
  * time           (time) datetime64[ns] 2022-01-01 ... 2022-01-01T08:00:00
  * photol_params  (photol_params) float64 1.0 2.0 3.0 4.0 ... 55.0 56.0 57.0
  * photol_coeffs  (photol_coeffs) int64 1 2 3 4 5 6 7 ... 90 91 92 93 94 95 96
  * therm_coeffs   (therm_coeffs) int64 1 2 3 4 5 6 ... 505 506 507 508 509 510
  * species        (species) <U10 'O1D' 'O' 'OH' ... 'CH3O2NO2' 'EMPOA' 'P2007'
    air_pressure   (level) float64 3.56e+04 3.31e+04 ... 2.263e+04 2.092e+04
    altitude       (level) float64 8.006e+03 8.55e+03 ... 1.114e+04 1.169e+04
  * longitude      (longitude) float64 0.0 2.5
  * latitude       (latitude) float64 0.0 2.5
  * level          (level) float64 356.0 331.0 307.4 285.2 ... 244.7 226.3 209.2
Data variables:
    local_time     (latitude, longitude, time) datetime64[ns] 2022-01-01 ... ...
    sza            (latitude, longitude, time) float64 2.748 2.673 ... 1.071
    J              (latitude, longitude, time, photol_params) float64 3.771e-...
    DJ             (latitude, longitude, level, time, photol_coeffs) float64 dask.array<chunksize=(2, 2, 8, 1, 96), meta=np.ndarray>
    RC             (latitude, longitude, level, time, therm_coeffs) float32 dask.array<chunksize=(2, 2, 8, 1, 510), meta=np.ndarray>
    soa            (latitude, longitude, level, time) float64 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
    mom            (latitude, longitude, level, time) float64 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
    BR01           (latitude, longitude, level, time) float64 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
    RO2            (latitude, longitude, level, time) float64 dask.array<chunksize=(2, 2, 8, 1), meta=np.ndarray>
    Y              (latitude, longitude, level, time, species) float64 dask.array<chunksize=(2, 2, 8, 1, 219), meta=np.ndarray>
    EM             (latitude, longitude, level, time, species) float64 dask.array<chunksize=(2, 2, 8, 1, 219), meta=np.ndarray>
    FL             (latitude, longitude, level, time, species) float64 dask.array<chunksize=(2, 2, 8, 1, 219), meta=np.ndarray>

In [16]:
emi = xr.Dataset(
    {
        "EM": (["latitude", "longitude", "level", "time", "species"],
                np.zeros((len(lats), len(lons), len(alts), len(chem.data.time), len(chem.data.species)))),
    },
    coords={
        "latitude": lats,
        "longitude": lons, 
        "level": alts,
        "time": chem.data.time,
        "species": chem.data.species,
    }
)

In [17]:
emi = ChemDataset(emi)
emi.data = emi.data.transpose("latitude", "longitude", "level", "time", ...)

In [18]:
boxm.eval(source=emi)

rc : 'f'-array(-1,-1,-1,-1,-1), not allocated 
dj : 'f'-array(-1,-1,-1,-1,-1), not allocated 
y : 'f'-array(-1,-1,-1,-1,-1), not allocated 
yp : 'f'-array(-1,-1,-1,-1,-1), not allocated 
em : 'f'-array(-1,-1,-1,-1,-1), not allocated 
fl : 'f'-array(-1,-1,-1,-1,-1), not allocated 
h2o : 'f'-array(-1,-1,-1), not allocated 
ro2 : 'f'-array(-1,-1,-1), not allocated 
o2 : 'f'-array(-1,-1,-1), not allocated 
p : 'f'-array(-1,-1,-1), not allocated 
l : 'f'-array(-1,-1,-1), not allocated 
m : 'f'-array(-1), not allocated 
dts : 'f'-scalar
ts : 'i'-scalar
deriv(ts)

Wrapper for ``deriv``.

Parameters
----------
ts : input float
chemco(ts)

Wrapper for ``chemco``.

Parameters
----------
ts : input float
photol(ts)

Wrapper for ``photol``.

Parameters
----------
ts : input float
calc_aerosol(ts)

Wrapper for ``calc_aerosol``.

Parameters
----------
ts : input float

<class 'numpy.ndarray'>
[[[[[0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    ...
    [0. 0. 0. ..